In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from numpy import asarray

In [ ]:
from datasets import load_dataset

# Load the GonzaloA/fake_news dataset
dataset = load_dataset('GonzaloA/fake_news')
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 24353
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
})


In [ ]:
train_data = dataset['train']
print(train_data)

Dataset({
    features: ['Unnamed: 0', 'title', 'text', 'label'],
    num_rows: 24353
})


In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df

,Unnamed: 0,title,text,label
0,0,‘Maury’ Show Official Facebook Posts F*CKED U...,Maury is perhaps one of the trashiest shows on...,0
1,1,Trump’s Favorite News Channel Tries To Soothe...,"Yesterday, after the father of one of the UCLA...",0
2,2,"Russia warns Iraq, Kurds not to destabilize Mi...",MOSCOW (Reuters) - Russia on Wednesday warned ...,1
3,3,WATCH STEVE SCALISE Throw A Strike At The Nati...,"House Majority Whip Steve Scalise (R., La.) th...",0
4,4,Trump Will HATE What Stephen Colbert Just Did...,It can be said that Late Show host Stephen Col...,0
...,...,...,...,...
24348,24348,EU Parliament chief asks Poland to ensure MEPs...,WARSAW (Reuters) - The president of the Europe...,1
24349,24349,Chemical weapons watchdog found sarin used in ...,AMSTERDAM/UNITED NATIONS (Reuters) - An inquir...,1
24350,24350,"Melissa Harris-Perry Is DONE With MSNBC, Pens...","As you may or may not know at this point, MSNB...",0
24351,24351,Trump's pick for Navy secretary withdraws,WASHINGTON (Reuters) - U.S. President Donald T...,1


In [ ]:
test_data = dataset['test']
print(test_data)

Dataset({
    features: ['Unnamed: 0', 'title', 'text', 'label'],
    num_rows: 8117
})


In [ ]:
df1 = pd.DataFrame(dataset['test'])

In [ ]:
df1

,Unnamed: 0,title,text,label
0,0,FORMER U.S. ATTORNEY: “It’s VERY Clear Intel C...,JOE DIGENOVA has been around D.C for decades a...,0
1,1,Rick Santorum Advises President To Quit Tweet...,"In a recent interview with CNN, former senator...",0
2,2,Stephen Colbert Explains The Horrifying Futur...,"With only three clowns left in GOP clown car, ...",0
3,3,Nepal votes in final round of polls at the end...,KATHMANDU (Reuters) - Nepalis began voting in ...,1
4,4,"More Americans think wealthy, not middle class...",WASHINGTON (Reuters) - Americans are more like...,1
...,...,...,...,...
8112,8112,Opposition leader says Brexit must not be used...,"BRUSSELS (Reuters) - Jeremy Corbyn, leader of ...",1
8113,8113,‘Pro-Life’ Scott Walker Just Signed Two Bills...,Republicans always talk a big game about being...,0
8114,8114,LIBERAL HUMOR? FLORIDA BAR POSTS SIGN Suggesti...,It s a federal offense to threaten a president...,0
8115,8115,New York protesters camp out at Goldman Sachs ...,NEW YORK (Reuters) - Dozens of protesters gath...,1


In [ ]:
validation_data = dataset['validation']
print(validation_data)

Dataset({
    features: ['Unnamed: 0', 'title', 'text', 'label'],
    num_rows: 8117
})


In [ ]:
df2 = pd.DataFrame(dataset['validation'])

In [ ]:
df2

,Unnamed: 0,title,text,label
0,0,Trump says to spend extra day in Philippines d...,WASHINGTON (Reuters) - President Donald Trump ...,1
1,1,Fire in Bangladesh textile factory kills six,DHAKA (Reuters) - A fire in a textile factory ...,1
2,2,Myanmar army drops charges against six journal...,YANGON (Reuters) - Myanmar s military on Frida...,1
3,3,Mentally-Ill Mother Shoots Sleeping Five-Year...,The warning signs were everywhere.Forty-one-ye...,0
4,4,No formal U.S. delegation to Castro memorial: ...,WASHINGTON (Reuters) - The United States will ...,1
...,...,...,...,...
8112,8112,Boiler Room #92 – The (Hollywood) Hills Have Eyes,Tune in to the Alternate Current Radio Network...,0
8113,8113,Putin: restriction on Russian media is attack ...,"DANANG, Vietnam (Reuters) - Russian President ...",1
8114,8114,REVENGE ATTACK ON MUSLIMS: VAN PLOWS INTO CROW...,A REVENGE ATTACK in London is the scene again ...,0
8115,8115,This Mainstream News Organization Thinks Trum...,We have always known that the Republican Party...,0


In [ ]:
import re

In [ ]:
import string

In [ ]:
#Lower case df
df['text'] = df['text'].str.lower()

In [ ]:
df

,Unnamed: 0,title,text,label
0,0,‘Maury’ Show Official Facebook Posts F*CKED U...,maury is perhaps one of the trashiest shows on...,0
1,1,Trump’s Favorite News Channel Tries To Soothe...,"yesterday, after the father of one of the ucla...",0
2,2,"Russia warns Iraq, Kurds not to destabilize Mi...",moscow (reuters) - russia on wednesday warned ...,1
3,3,WATCH STEVE SCALISE Throw A Strike At The Nati...,"house majority whip steve scalise (r., la.) th...",0
4,4,Trump Will HATE What Stephen Colbert Just Did...,it can be said that late show host stephen col...,0
...,...,...,...,...
24348,24348,EU Parliament chief asks Poland to ensure MEPs...,warsaw (reuters) - the president of the europe...,1
24349,24349,Chemical weapons watchdog found sarin used in ...,amsterdam/united nations (reuters) - an inquir...,1
24350,24350,"Melissa Harris-Perry Is DONE With MSNBC, Pens...","as you may or may not know at this point, msnb...",0
24351,24351,Trump's pick for Navy secretary withdraws,washington (reuters) - u.s. president donald t...,1


In [ ]:
#Lower case df1
df1['text'] = df1['text'].str.lower()

In [ ]:
df1

,Unnamed: 0,title,text,label
0,0,FORMER U.S. ATTORNEY: “It’s VERY Clear Intel C...,joe digenova has been around d.c for decades a...,0
1,1,Rick Santorum Advises President To Quit Tweet...,"in a recent interview with cnn, former senator...",0
2,2,Stephen Colbert Explains The Horrifying Futur...,"with only three clowns left in gop clown car, ...",0
3,3,Nepal votes in final round of polls at the end...,kathmandu (reuters) - nepalis began voting in ...,1
4,4,"More Americans think wealthy, not middle class...",washington (reuters) - americans are more like...,1
...,...,...,...,...
8112,8112,Opposition leader says Brexit must not be used...,"brussels (reuters) - jeremy corbyn, leader of ...",1
8113,8113,‘Pro-Life’ Scott Walker Just Signed Two Bills...,republicans always talk a big game about being...,0
8114,8114,LIBERAL HUMOR? FLORIDA BAR POSTS SIGN Suggesti...,it s a federal offense to threaten a president...,0
8115,8115,New York protesters camp out at Goldman Sachs ...,new york (reuters) - dozens of protesters gath...,1


In [ ]:
#Lower case df2
df2['text'] = df2['text'].str.lower()

In [ ]:
df2

,Unnamed: 0,title,text,label
0,0,Trump says to spend extra day in Philippines d...,washington (reuters) - president donald trump ...,1
1,1,Fire in Bangladesh textile factory kills six,dhaka (reuters) - a fire in a textile factory ...,1
2,2,Myanmar army drops charges against six journal...,yangon (reuters) - myanmar s military on frida...,1
3,3,Mentally-Ill Mother Shoots Sleeping Five-Year...,the warning signs were everywhere.forty-one-ye...,0
4,4,No formal U.S. delegation to Castro memorial: ...,washington (reuters) - the united states will ...,1
...,...,...,...,...
8112,8112,Boiler Room #92 – The (Hollywood) Hills Have Eyes,tune in to the alternate current radio network...,0
8113,8113,Putin: restriction on Russian media is attack ...,"danang, vietnam (reuters) - russian president ...",1
8114,8114,REVENGE ATTACK ON MUSLIMS: VAN PLOWS INTO CROW...,a revenge attack in london is the scene again ...,0
8115,8115,This Mainstream News Organization Thinks Trum...,we have always known that the republican party...,0


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
#Tokenize data df
from nltk.tokenize import word_tokenize

df['tokens'] = df['text'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df[['text', 'tokens']].head())


                                                text  \
0  maury is perhaps one of the trashiest shows on...   
1  yesterday, after the father of one of the ucla...   
2  moscow (reuters) - russia on wednesday warned ...   
3  house majority whip steve scalise (r., la.) th...   
4  it can be said that late show host stephen col...   

                                              tokens  
0  [maury, is, perhaps, one, of, the, trashiest, ...  
1  [yesterday, ,, after, the, father, of, one, of...  
2  [moscow, (, reuters, ), -, russia, on, wednesd...  
3  [house, majority, whip, steve, scalise, (, r.,...  
4  [it, can, be, said, that, late, show, host, st...  


In [ ]:
#Tokenize data df1
from nltk.tokenize import word_tokenize

df1['tokens'] = df1['text'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df1[['text', 'tokens']].head())

                                                text  \
0  joe digenova has been around d.c for decades a...   
1  in a recent interview with cnn, former senator...   
2  with only three clowns left in gop clown car, ...   
3  kathmandu (reuters) - nepalis began voting in ...   
4  washington (reuters) - americans are more like...   

                                              tokens  
0  [joe, digenova, has, been, around, d.c, for, d...  
1  [in, a, recent, interview, with, cnn, ,, forme...  
2  [with, only, three, clowns, left, in, gop, clo...  
3  [kathmandu, (, reuters, ), -, nepalis, began, ...  
4  [washington, (, reuters, ), -, americans, are,...  


In [ ]:
#Tokenize data df2
from nltk.tokenize import word_tokenize

df2['tokens'] = df2['text'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df2[['text', 'tokens']].head())

                                                text  \
0  washington (reuters) - president donald trump ...   
1  dhaka (reuters) - a fire in a textile factory ...   
2  yangon (reuters) - myanmar s military on frida...   
3  the warning signs were everywhere.forty-one-ye...   
4  washington (reuters) - the united states will ...   

                                              tokens  
0  [washington, (, reuters, ), -, president, dona...  
1  [dhaka, (, reuters, ), -, a, fire, in, a, text...  
2  [yangon, (, reuters, ), -, myanmar, s, militar...  
3  [the, warning, signs, were, everywhere.forty-o...  
4  [washington, (, reuters, ), -, the, united, st...  


In [ ]:
# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
#Function to remove stopwords
def remove_stopwords(data):
    return ' '.join(word for word in data.split() if word.lower() not in stop_words)

df['text'] = df['text'].apply(remove_stopwords)
df1['text'] = df1['text'].apply(remove_stopwords)
df2['text'] = df2['text'].apply(remove_stopwords)

In [ ]:
print(df.head(5))
print(df1.head(5))
print(df1.head(5))

   Unnamed: 0                                              title  \
0           0   ‘Maury’ Show Official Facebook Posts F*CKED U...   
1           1   Trump’s Favorite News Channel Tries To Soothe...   
2           2  Russia warns Iraq, Kurds not to destabilize Mi...   
3           3  WATCH STEVE SCALISE Throw A Strike At The Nati...   
4           4   Trump Will HATE What Stephen Colbert Just Did...   

                                                text  label  \
0  maury perhaps one trashiest shows television t...      0   
1  yesterday, father one ucla players arrested ch...      0   
2  moscow (reuters) - russia wednesday warned ira...      1   
3  house majority whip steve scalise (r., la.) th...      0   
4  said late show host stephen colbert clearly gi...      0   

                                              tokens  
0  [maury, is, perhaps, one, of, the, trashiest, ...  
1  [yesterday, ,, after, the, father, of, one, of...  
2  [moscow, (, reuters, ), -, russia, on, wedne

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Tokenize data
from nltk.tokenize import word_tokenize

df['tokens'] = df['text'].apply(word_tokenize)

In [ ]:
# Concatenate train and validation data for training
X_train = pd.concat([df['text'], df1['text']], ignore_index=True)
y_train = pd.concat([df['label'], df1['label']], ignore_index=True)

# Test data
X_test = df2['text']
y_test = df2['label']

In [ ]:
#Split in train, test and validation
from sklearn.model_selection import train_test_split

# Split data into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
#Making x_train and y_train columns
X_train = train_df['text']
y_train = train_df['text']

In [ ]:
!pip install transformers
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def encode_text(text):
    # Tokenize the text
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt", max_length=512)

    # Obtain the input IDs from the tokenizer output
    input_ids = inputs.input_ids

    with torch.no_grad():
        # Forward pass through the model
        outputs = model(input_ids)

    # Get the last hidden state of the [CLS] token
    cls_embedding = outputs.last_hidden_state[:, 0, :]

    return cls_embedding.numpy()


In [ ]:
# Apply encoding to the train and test data
X_train_encoded = [encode_text(text) for text in X_train]
X_test_encoded = [encode_text(text) for text in X_test]

#X_train_encoded.to_csv("X_train_encoded.csv")
#X_test_encoded.to_csv("X_test_encoded.csv")
import pandas as pd
# Convert lists to DataFrames
df_train = pd.DataFrame(X_train_encoded)
df_test = pd.DataFrame(X_test_encoded)

# Save DataFrames to CSV files
df_train.to_csv("X_train_encoded.csv")
df_test.to_csv("X_test_encoded.csv")


In [ ]:
# Train Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train_encoded, y_train)

# Make predictions on train set
logreg_train_preds = logreg_model.predict(X_train_encoded)

# Evaluate Logistic Regression model on train set
logreg_train_scores = {
    "accuracy": accuracy_score(y_train, logreg_train_preds),
    "precision": precision_score(y_train, logreg_train_preds, average='weighted'),
    "recall": recall_score(y_train, logreg_train_preds, average='weighted'),
    "f1": f1_score(y_train, logreg_train_preds, average='weighted')
}
print("Logistic Regression Scores:")
print("Accuracy:", logreg_train_scores["accuracy"])
print("Precision:", logreg_train_scores["precision"])
print("Recall:", logreg_train_scores["recall"])
print("F1-Score:", logreg_train_scores["f1"])
print()

In [ ]:
# Train SVM model
svm_model = SVC()
svm_model.fit(X_train_encoded, y_train)

# Make predictions on train set
svm_train_preds = svm_model.predict(X_train_encoded)

# Evaluate SVM model on train set
svm_train_scores = {
    "accuracy": accuracy_score(y_train, svm_train_preds),
    "precision": precision_score(y_train, svm_train_preds, average='weighted'),
    "recall": recall_score(y_train, svm_train_preds, average='weighted'),
    "f1": f1_score(y_train, svm_train_preds, average='weighted')
}
print("SVM Scores:")
print("Accuracy:", svm_train_scores["accuracy"])
print("Precision:", svm_train_scores["precision"])
print("Recall:", svm_train_scores["recall"])
print("F1-Score:", svm_train_scores["f1"])
print()

In [ ]:

# Train Neural Network model
nn_model = MLPClassifier(hidden_layer_sizes=(768,2), max_iter=100, early_stopping=True, validation_fraction=0.2)
nn_model.fit(X_train_encoded, y_train)
# Make predictions on train set
nn_train_preds = nn_model.predict(X_train_encoded)

# Evaluate Neural Network model on train set
nn_train_scores = {
    "accuracy": accuracy_score(y_train, nn_train_preds),
    "precision": precision_score(y_train, nn_train_preds, average='weighted'),
    "recall": recall_score(y_train, nn_train_preds, average='weighted'),
    "f1": f1_score(y_train, nn_train_preds, average='weighted')
}
print("Neural Network Scores:")
print("Accuracy:", nn_train_scores["accuracy"])
print("Precision:", nn_train_scores["precision"])
print("Recall:", nn_train_scores["recall"])
print("F1-Score:", nn_train_scores["f1"])
print()

In [ ]:
import numpy as np
# Reshape X_train_encoded to add another dimension
X_train_encoded_reshaped = np.expand_dims(X_train_encoded, axis=-1)

In [ ]:
# Train LSTM model
lstm_model = Sequential()
lstm_model.add(Dense(64, input_shape=(768,), activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_encoded_reshaped[:50], y_train[:50], epochs=10, batch_size=32)

# Make predictions on train set
lstm_train_preds = (lstm_model.predict(X_train_encoded_reshaped) > 0.5).astype(int).flatten()

# Evaluate LSTM model on train set
lstm_train_scores = {
    "accuracy": accuracy_score(y_train, lstm_train_preds),
    "precision": precision_score(y_train, lstm_train_preds, average='weighted'),
    "recall": recall_score(y_train, lstm_train_preds, average='weighted'),
    "f1": f1_score(y_train, lstm_train_preds, average='weighted')
}
print("LSTM Scores:")
print("Accuracy:", lstm_train_scores["accuracy"])
print("Precision:", lstm_train_scores["precision"])
print("Recall:", lstm_train_scores["recall"])
print("F1-Score:", lstm_train_scores["f1"])
print()

In [ ]:
#TEST
# Test Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_test_encoded, y_test)

# Make predictions on train set
logreg_test_preds = logreg_model.predict(X_test_encoded)

# Evaluate Logistic Regression model on test set
logreg_test_scores = {
    "accuracy": accuracy_score(y_test, logreg_test_preds),
    "precision": precision_score(y_test, logreg_test_preds, average='weighted'),
    "recall": recall_score(y_test, logreg_test_preds, average='weighted'),
    "f1": f1_score(y_test, logreg_test_preds, average='weighted')
}
print("Logistic Regression Scores:")
print("Accuracy:", logreg_test_scores["accuracy"])
print("Precision:", logreg_test_scores["precision"])
print("Recall:", logreg_test_scores["recall"])
print("F1-Score:", logreg_test_scores["f1"])
print()

In [ ]:
# Test SVM model
svm_model = SVC()
svm_model.fit(X_test_encoded, y_test)

# Make predictions on test set
svm_test_preds = svm_model.predict(X_test_encoded)

# Evaluate SVM model on test set
svm_test_scores = {
    "accuracy": accuracy_score(y_test, svm_test_preds),
    "precision": precision_score(y_test, svm_test_preds, average='weighted'),
    "recall": recall_score(y_test, svm_test_preds, average='weighted'),
    "f1": f1_score(y_test, svm_test_preds, average='weighted')
}
print("SVM Scores:")
print("Accuracy:", svm_test_scores["accuracy"])
print("Precision:", svm_test_scores["precision"])
print("Recall:", svm_test_scores["recall"])
print("F1-Score:", svm_test_scores["f1"])
print()

In [ ]:

# Test Neural Network model
nn_model = MLPClassifier(hidden_layer_sizes=(768,2), max_iter=100, early_stopping=True, validation_fraction=0.2)
nn_model.fit(X_test_encoded, y_test)
# Make predictions on test set
nn_test_preds = nn_model.predict(X_test_encoded)

# Evaluate Neural Network model on test set
nn_test_scores = {
    "accuracy": accuracy_score(y_test, nn_test_preds),
    "precision": precision_score(y_test, nn_test_preds, average='weighted'),
    "recall": recall_score(y_test, nn_test_preds, average='weighted'),
    "f1": f1_score(y_test, nn_test_preds, average='weighted')
}
print("Neural Network Scores:")
print("Accuracy:", nn_test_scores["accuracy"])
print("Precision:", nn_test_scores["precision"])
print("Recall:", nn_test_scores["recall"])
print("F1-Score:", nn_test_scores["f1"])
print()

In [ ]:
import numpy as np
# Reshape X_train_encoded to add another dimension
X_test_encoded_reshaped = np.expand_dims(X_test_encoded, axis=-1)

In [ ]:
# Test LSTM model
lstm_model = Sequential()
lstm_model.add(Dense(64, input_shape=(768,), activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_test_encoded_reshaped[:50], y_test[:50], epochs=10, batch_size=32)

# Make predictions on test set
lstm_test_preds = (lstm_model.predict(X_test_encoded_reshaped) > 0.5).astype(int).flatten()

# Evaluate LSTM model on test set
lstm_test_scores = {
    "accuracy": accuracy_score(y_test, lstm_test_preds),
    "precision": precision_score(y_test, lstm_test_preds, average='weighted'),
    "recall": recall_score(y_test, lstm_test_preds, average='weighted'),
    "f1": f1_score(y_test, lstm_test_preds, average='weighted')
}
print("LSTM Scores:")
print("Accuracy:", lstm_test_scores["accuracy"])
print("Precision:", lstm_test_scores["precision"])
print("Recall:", lstm_test_scores["recall"])
print("F1-Score:", lstm_test_scores["f1"])
print()